In [ ]:
import os
import h5py
import tensorflow as tf
import numpy as np

from time import strftime
from scipy.signal import spectrogram

In [ ]:
MODEL_NAME = "ClassificatorPlayground"

In [ ]:
def get_data(file_path, idx_start, idx_end, idx_slice):
    x_train = None
    y_train = None
    x_test = None
    y_test = None
    with h5py.File(file_path, "r") as f:
        x_train = f["data"][idx_start:idx_slice]
        y_train = f["labels"][idx_start:idx_slice]
        x_test = f["data"][idx_slice:idx_end]
        y_test = f["labels"][idx_slice:idx_end]
        return (x_train, y_train, x_test, y_test)

In [ ]:
(x_train, y_train, x_test, y_test) = get_data("../data/LEN-DB-processed.hdf5", 10000, 20000, 18000)

In [ ]:
x_train *= 1000000
x_test *= 1000000

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 1620)
x_test = x_test.reshape(x_test.shape[0], 1620)

In [ ]:
X = np.expand_dims(x_train, axis = 2)
X.shape

In [ ]:
Y = np.array(y_train)
Y.shape

In [ ]:
T = np.expand_dims(x_test, axis = 2)
T.shape

In [ ]:
Q = np.array(y_test)
Q.shape

In [ ]:
folder_name = f"{MODEL_NAME} at {strftime('%H:%M')}"
log_dir = os.path.join("../log/", folder_name)

try:
    os.makedirs(log_dir)
except OSError as exception:
    print(exception.strerror)
else:
    print("Successfully created dirs!")

In [ ]:
drop_out_rate = 0.2

model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(1620,1)),
    tf.keras.layers.Conv1D(8, 11, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Conv1D(16, 7, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Conv1D(32, 5, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Conv1D(64, 5, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Conv1D(128, 3, padding='valid', activation='relu', strides=1),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(256, activation='relu'),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(drop_out_rate),
    tf.keras.layers.Dense(2, activation='softmax')
])

In [ ]:
adam = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=adam,
              loss=tf.keras.losses.SparseCategoricalCrossentropy(),
              metrics=['accuracy'])
model.summary()

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)

In [ ]:
model.fit(X, Y, epochs=25, validation_data=(T, Q), callbacks=[tensorboard_callback])

In [ ]:
test_loss, test_acc = model.evaluate(T, Q, verbose=2)